In [1]:
from tqdm.notebook import tqdm
from IPython.display import display, HTML

import numpy as np
import pandas as pd

import plotly.graph_objects as go

import datetime
from pathlib import Path

import intake

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.utils import shuffle
import itertools

from tensorflow.keras.regularizers import l2
import tensorflow as tf
from tensorflow.keras import backend as K

from tensorflow.keras.layers import ConvLSTM2D, Conv2D, Conv3D, Dense, TimeDistributed, MaxPooling2D 
from tensorflow.keras.layers import Activation, Dropout, Flatten, BatchNormalization, Input
from tensorflow.keras.layers import LSTM, GlobalAveragePooling3D, GlobalAveragePooling2D
from tensorflow.keras.layers import GlobalAveragePooling1D

2024-05-21 20:04:16.766971: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-21 20:04:17.641236: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
catalog = intake.open_catalog(Path('../catalog.yml'))
source = getattr(catalog, 'treesat_multi')
df = source.read()

In [3]:
selected_bands = [f'B{x}' for x in range(2, 9)] + ['B8A', 'B11', 'B12', 'TCI_R', 'TCI_G', 'TCI_B']

In [4]:
target = source.metadata['categories']['multi'] # multi / trinary
labels = df[target].to_numpy()

indexes = np.arange(labels.shape[0])
shuffled_indexes = shuffle(indexes, random_state=42)

shuffled_labels = labels[shuffled_indexes]

shuffled_years = df['YEAR'].astype(int).to_numpy()[shuffled_indexes]
shuffled_years = np.where(shuffled_years < 2017, 2017, shuffled_years)

model_dir = Path('models').joinpath('seasons')
model_dir.mkdir(parents=True, exist_ok=True)

data_dir = Path('seasonal_median')

In [5]:
def get_training_features(years, seasons, data_dir, shuffled_years):
    selected_data = []
    for season in seasons:
        year_data = np.zeros((shuffled_years.shape[0], 6, 6, 13))

        for year in years:
            year_indices = np.where(shuffled_years == year)[0]

            filepath = data_dir.joinpath(f'processed_treesat_{year}{str(season).zfill(2)}.npy')
            with open(filepath, 'rb') as f:
                year_data[year_indices, ...] = np.load(f)[year_indices]
        selected_data.append(year_data)
        
    return np.stack(selected_data, axis=1)

In [6]:
class KerasModelCreator:
    def normalise_X(self, X, p=1):
        upper = np.percentile(X, 100-p)
        lower = np.percentile(X, p)
    
        bounded_X = np.where(X > upper, np.median(X), X)
        bounded_X = np.where(X < lower, np.median(X), bounded_X)
        
        scaled_X = (bounded_X - lower)/(upper - lower)
        return scaled_X

    def run(self, X, y, model_path, epochs=10, overwrite=False):
        ''' 
        If not overwrite and there's an existing model, the model will 
        continue trainingw if the given epoch is bigger than the previous,
        else just evaluate.
        Ensure train splits are the same across continuations / evaluations
        by not modifying the random_state in split_and_normalise.
        '''
        model_savepoint = model_path.parent.joinpath(model_path.stem)
        log_file = model_path.with_suffix('.log')

        if overwrite:
            for f in [model_path, log_file] + list(model_savepoint.glob('*')):
                f.unlink(missing_ok=True)
                
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.1, random_state=42, shuffle=False)
        
        for i in range(X_train.shape[-1]):
            X_train[...,i] = self.normalise_X(X_train[...,i])
            X_test[...,i] = self.normalise_X(X_test[...,i])
        
        default_metrics = ['accuracy', 'root_mean_squared_error', 'r2_score']
        
        if model_path.is_file():
            model = tf.keras.models.load_model(model_path)
        else:
            model = self.build_model(y.shape[1], X_train.shape[1:], default_metrics)

        callbacks = [
            tf.keras.callbacks.BackupAndRestore(
                model_savepoint, save_freq='epoch', delete_checkpoint=False
            ),
            tf.keras.callbacks.CSVLogger(
                log_file, append=True
            )
        ]

        if log_file.is_file():
            df = pd.read_csv(log_file)[['epoch', 'loss'] + default_metrics]
            df['epoch'] += 1
            print('Previous training:')
                
            display(HTML(df.to_html(index=False)))
        
        model.fit(
            X_train, y_train, epochs=epochs, verbose=1, batch_size=10, callbacks=callbacks, 
            shuffle=False, validation_data=(X_test, y_test)
        )

        model.save(model_path)

        
    def build_model(self, output_shape, input_shape, metrics):
        m = tf.keras.Sequential()
        
        m.add(Input(input_shape))

        m.add(ConvLSTM2D(
            filters=32, kernel_size=2, 
            padding='same', activation='tanh',
            return_sequences=True
        ))

        m.add(Conv3D(
            filters=16, kernel_size=2, 
            padding='same'
        ))

        # m.add(GlobalAveragePooling2D())
        m.add(Flatten())
        
        m.add(Dense(
            output_shape, 
            activation='softmax', 
        ))
        m.compile(optimizer='rmsprop', loss='mse', metrics=metrics)
        
        return m

In [7]:
train_years = [2017, 2018, 2019, 2020]
all_seasons = [3, 6, 9, 12]

model_name = f'conv_seasons_{"_".join(map(str, all_seasons))}_years_{"_".join(map(str, train_years))}.keras'
model_path = model_dir.joinpath(model_name)

train_features = get_training_features(train_years, all_seasons, data_dir, shuffled_years)

In [8]:
train_features.shape

(50381, 4, 6, 6, 13)

In [ ]:
result = KerasModelCreator().run(
    train_features,
    shuffled_labels,
    model_path, epochs=20, overwrite=True
)

Epoch 1/20
4535/4535 ━━━━━━━━━━━━━━━━━━━━ 29s 6ms/step - accuracy: 0.1573 - loss: 0.0488 - r2_score: -0.0145 - root_mean_squared_error: 0.2209 - val_accuracy: 0.1725 - val_loss: 0.0488 - val_r2_score: -0.0010 - val_root_mean_squared_error: 0.2209
Epoch 2/20
4535/4535 ━━━━━━━━━━━━━━━━━━━━ 29s 6ms/step - accuracy: 0.1674 - loss: 0.0485 - r2_score: -6.3600e-04 - root_mean_squared_error: 0.2203 - val_accuracy: 0.1683 - val_loss: 0.0488 - val_r2_score: -1.1264e-04 - val_root_mean_squared_error: 0.2208
Epoch 3/20
4535/4535 ━━━━━━━━━━━━━━━━━━━━ 29s 6ms/step - accuracy: 0.1704 - loss: 0.0485 - r2_score: 4.4402e-04 - root_mean_squared_error: 0.2202 - val_accuracy: 0.1661 - val_loss: 0.0487 - val_r2_score: 3.0641e-04 - val_root_mean_squared_error: 0.2208
Epoch 4/20
4535/4535 ━━━━━━━━━━━━━━━━━━━━ 29s 7ms/step - accuracy: 0.1726 - loss: 0.0484 - r2_score: 0.0011 - root_mean_squared_error: 0.2201 - val_accuracy: 0.1677 - val_loss: 0.0487 - val_r2_score: 6.3957e-04 - val_root_mean_squared_error: 0.2

In [ ]:
tf.keras.models.load_model(model_path).summary()

In [ ]:
# import subprocess
# subprocess.run(['sudo', 'shutdown', 'now'])

In [ ]:
# season_combinations = itertools.chain.from_iterable(
#     itertools.combinations(all_seasons, r) for r in range(1, len(all_seasons)+1))
# season_combinations = list(season_combinations)